# Clean Water AI - Environmental Data Collection

This notebook collects environmental data at exact water point locations for spatially and temporally accurate analysis.

## Setup and Data Loading

In [17]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
from scripts.extract_data import get_wpdx_kenya , collapse_zinc_csv, merge_mercury_zinc, collapse_mercury_csv
warnings.filterwarnings("ignore")


In [19]:
import geemap
import ee

# Authenticate and initialize Earth Engine with Google Cloud credentials and account
ee.Authenticate()
ee.Initialize()

In [23]:
zinc_df = collapse_zinc_csv()
zinc_df = collapse_mercury_csv()
zinc_df.head()
zinc_df.info()
merge_mercury_zinc()

✅ || Collapsed 203912 rows to 177882 rows and saved to ../data/raw/Zinc_collapsed.csv
✅ || Collapsed 229612 rows to 216637 rows and saved to ../data/raw/Mercury_collapsed.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216637 entries, 0 to 216636
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   GEMS.Station.Number   216637 non-null  object 
 1   Sample.Date           216637 non-null  object 
 2   Sample.Time           216637 non-null  object 
 3   Depth                 216637 non-null  float64
 4   Analysis.Method.Code  216637 non-null  object 
 5   Value.Flags           115985 non-null  object 
 6   Unit                  216637 non-null  object 
 7   Data.Quality          216637 non-null  object 
 8   Value.Dissolved       50849 non-null   float64
 9   Value.Total           179565 non-null  float64
dtypes: float64(3), object(7)
memory usage: 16.5+ MB


,GEMS.Station.Number,Sample.Date,Sample.Time,Depth,Analysis.Method.Code,Value.Flags,Unit,Data.Quality,Value.Dissolved,Value.Total
0,ARG00014,2011-09-06,11:30,0.3,0,<,mg/l,Fair,0.040,0.040
1,ARG00014,2012-03-14,12:20,0.3,0,NaN,mg/l,Fair,0.044,0.348
2,ARG00014,2012-06-06,13:20,0.3,0,<,mg/l,Fair,0.040,0.185
3,ARG00014,2012-09-18,08:50,0.3,0,<,mg/l,Fair,0.040,0.100
4,ARG00014,2012-12-12,10:00,0.3,0,<,mg/l,Fair,40.000,0.348
...,...,...,...,...,...,...,...,...,...,...
394514,USA00021,1991-06-18,09:00,0.3,FAAS-APr-D,NaN,mg/l,Fair,0.101,0.300
394515,USA00021,1991-07-29,09:30,0.3,FAAS-APr-D,NaN,mg/l,Fair,0.014,NaN
394516,USA00021,1991-08-08,07:00,0.3,FAAS-APr-D,<,mg/l,Fair,0.006,0.010
394517,USA00021,1991-09-30,10:00,0.3,FAAS-APr-D-NA,<,mg/l,Fair,NaN,0.010


In [ ]:
df_wpdx = get_wpdx_kenya()


# Convert report_date to datetime and create date ranges for environmental data
df_wpdx['report_date'] = pd.to_datetime(df_wpdx['report_date'])
df_wpdx['env_start_date'] = df_wpdx['report_date'] - timedelta(days=30)
df_wpdx['env_end_date'] = df_wpdx['report_date']

# Sample subset for processing
sample_size = 1000 # min(500, len(kenya_points))
kenya_sample = df_wpdx.sample(n=sample_size, random_state=42).reset_index(drop=True)

print(f"Processing {len(kenya_sample)} water points with temporal accuracy")

✅ || wpdx_kenya.csv already exists with 21953 rows, skipping fetch.
Processing 1000 water points with temporal accuracy


## Create Earth Engine FeatureCollection

In [ ]:
# Create Earth Engine points from water point coordinates with date info
def create_ee_feature(row):
    point = ee.Geometry.Point([row['lon_deg'], row['lat_deg']])
    return ee.Feature(point, {
        'wpdx_id': str(row['wpdx_id']),
        'point_id': int(row.name),
        'start_date': row['env_start_date'].strftime('%Y-%m-%d'),
        'end_date': row['env_end_date'].strftime('%Y-%m-%d')
    })

water_points_fc = ee.FeatureCollection([create_ee_feature(row) for _, row in kenya_sample.iterrows()])
print("Water points FeatureCollection created with temporal info")

Water points FeatureCollection created with temporal info


## CHIRPS Rainfall Data

In [ ]:
# Sample CHIRPS rainfall at water point locations with temporal accuracy
def sample_chirps_for_point(feature):
    start_date = feature.get('start_date')
    end_date = feature.get('end_date')
    
    chirps_img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
        .filterDate(start_date, end_date) \
        .sum() \
        .rename('rainfall_30d_mm')
    
    return chirps_img.sampleRegions(
        collection=ee.FeatureCollection([feature]),
        scale=5000,
        geometry=True
    ).first().copyProperties(feature)

chirps_sampled = water_points_fc.map(sample_chirps_for_point)

# Convert to DataFrame
chirps_list = chirps_sampled.getInfo()['features']
chirps_data = []

for feature in chirps_list:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    chirps_data.append({
        'wpdx_id': props.get('wpdx_id'),
        'point_id': props.get('point_id'),
        'longitude': coords[0],
        'latitude': coords[1],
        'rainfall_30d_mm': props.get('rainfall_30d_mm'),
        'start_date': props.get('start_date'),
        'end_date': props.get('end_date')
    })

df_chirps = pd.DataFrame(chirps_data)
df_chirps.to_csv("chirps_rainfall_wpdx_points.csv", index=False)
print(f"CHIRPS data collected: {len(df_chirps)} points with temporal accuracy")
df_chirps.head()

TypeError: Image.sampleRegions() got an unexpected keyword argument 'geometry'

## MODIS NDVI Data

In [ ]:
# Sample MODIS NDVI with all bands at water point locations with temporal accuracy
def sample_ndvi_for_point(feature):
    start_date = feature.get('start_date')
    end_date = feature.get('end_date')
    
    ndvi_collection = ee.ImageCollection("MODIS/061/MOD13Q1") \
        .filterDate(start_date, end_date) \
        .select(['DayOfYear', 'DetailedQA', 'EVI', 'NDVI', 'RelativeAzimuth', 
                 'SolarZenith', 'SummaryQA', 'ViewZenith', 'sur_refl_b01', 
                 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']) \
        .mean()

    # Apply proper scaling factors
    ndvi_scaled = ndvi_collection.select(['NDVI', 'EVI']).multiply(0.0001) \
        .addBands(ndvi_collection.select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']).multiply(0.0001)) \
        .addBands(ndvi_collection.select(['RelativeAzimuth', 'SolarZenith', 'ViewZenith']).multiply(0.01)) \
        .addBands(ndvi_collection.select(['DayOfYear', 'DetailedQA', 'SummaryQA']))

    return ndvi_scaled.sampleRegions(
        collection=ee.FeatureCollection([feature]),
        scale=250,
        geometry=True
    ).first().copyProperties(feature)

ndvi_sampled = water_points_fc.map(sample_ndvi_for_point)

# Convert to DataFrame
ndvi_list = ndvi_sampled.getInfo()['features']
ndvi_data = []

for feature in ndvi_list:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    ndvi_data.append({
        'wpdx_id': props.get('wpdx_id'),
        'point_id': props.get('point_id'),
        'longitude': coords[0],
        'latitude': coords[1],
        'DayOfYear': props.get('DayOfYear'),
        'DetailedQA': props.get('DetailedQA'),
        'EVI': props.get('EVI'),
        'NDVI': props.get('NDVI'),
        'RelativeAzimuth': props.get('RelativeAzimuth'),
        'SolarZenith': props.get('SolarZenith'),
        'SummaryQA': props.get('SummaryQA'),
        'ViewZenith': props.get('ViewZenith'),
        'sur_refl_b01': props.get('sur_refl_b01'),
        'sur_refl_b02': props.get('sur_refl_b02'),
        'sur_refl_b03': props.get('sur_refl_b03'),
        'sur_refl_b07': props.get('sur_refl_b07'),
    'start_date': props.get('start_date'),
    'end_date': props.get('end_date')
})

df_ndvi = pd.DataFrame(ndvi_data)
df_ndvi.to_csv("modis_ndvi_all_bands_wpdx_points.csv", index=False)
print(f"MODIS data collected: {len(df_ndvi)} points with all bands and temporal accuracy")
df_ndvi.head()

## MODIS LST Data

In [ ]:
# Sample MODIS LST at water point locations with temporal accuracy
def sample_lst_for_point(feature):
    start_date = feature.get('start_date')
    end_date = feature.get('end_date')
    
    lst_img = ee.ImageCollection("MODIS/061/MOD11A2") \
        .filterDate(start_date, end_date) \
        .select('LST_Day_1km') \
        .mean() \
        .multiply(0.02) \
        .subtract(273.15) \
        .rename('lst_celsius')

    return lst_img.sampleRegions(
        collection=ee.FeatureCollection([feature]),
        scale=1000,
        geometry=True
    ).first().copyProperties(feature)

lst_sampled = water_points_fc.map(sample_lst_for_point)

# Convert to DataFrame
lst_list = lst_sampled.getInfo()['features']
lst_data = []

for feature in lst_list:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    lst_data.append({
        'wpdx_id': props.get('wpdx_id'),
        'point_id': props.get('point_id'),
        'longitude': coords[0],
        'latitude': coords[1],
        'lst_celsius': props.get('lst_celsius'),
        'start_date': props.get('start_date'),
        'end_date': props.get('end_date')
    })

df_lst = pd.DataFrame(lst_data)
df_lst.to_csv("modis_lst_wpdx_points.csv", index=False)
print(f"LST data collected: {len(df_lst)} points with temporal accuracy")
df_lst.head()

## SMAP Soil Moisture Data

In [ ]:
# Sample SMAP soil moisture at water point locations with temporal accuracy
def sample_smap_for_point(feature):
    start_date = feature.get('start_date')
    end_date = feature.get('end_date')
    
    smap_img = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006") \
        .filterDate(start_date, end_date) \
        .select(['soil_moisture_am', 'soil_moisture_pm']) \
        .mean()

    return smap_img.sampleRegions(
        collection=ee.FeatureCollection([feature]),
        scale=9000,
        geometry=True
    ).first().copyProperties(feature)

smap_sampled = water_points_fc.map(sample_smap_for_point)

# Convert to DataFrame
smap_list = smap_sampled.getInfo()['features']
smap_data = []

for feature in smap_list:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    smap_data.append({
        'wpdx_id': props.get('wpdx_id'),
        'point_id': props.get('point_id'),
        'longitude': coords[0],
        'latitude': coords[1],
        'soil_moisture_am': props.get('soil_moisture_am'),
        'soil_moisture_pm': props.get('soil_moisture_pm'),
        'start_date': props.get('start_date'),
        'end_date': props.get('end_date')
    })

df_smap = pd.DataFrame(smap_data)
df_smap.to_csv("smap_soil_moisture_wpdx_points.csv", index=False)
print(f"SMAP data collected: {len(df_smap)} points with temporal accuracy")
df_smap.head()

## WorldPop Population Data

In [ ]:
# Sample WorldPop population density (using 2020 as baseline since it's demographic data)
worldpop_img = ee.ImageCollection("WorldPop/GP/100m/pop") \
    .filterDate('2020-01-01', '2020-12-31') \
    .mosaic() \
    .rename('pop_density')

worldpop_sampled = worldpop_img.sampleRegions(
    collection=water_points_fc,
    scale=100,
    geometries=True
)

# Convert to DataFrame
worldpop_list = worldpop_sampled.getInfo()['features']
worldpop_data = []

for feature in worldpop_list:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    worldpop_data.append({
        'wpdx_id': props.get('wpdx_id'),
        'point_id': props.get('point_id'),
        'longitude': coords[0],
        'latitude': coords[1],
        'pop_density': props.get('pop_density')
    })

df_worldpop = pd.DataFrame(worldpop_data)
df_worldpop.to_csv("worldpop_wpdx_points.csv", index=False)
print(f"WorldPop data collected: {len(df_worldpop)} points")
df_worldpop.head()

## Merge All Environmental Data

In [ ]:
# Merge all environmental datasets with water point data
df_merged = df_chirps[['wpdx_id', 'point_id', 'longitude', 'latitude', 'rainfall_30d_mm']] \
    .merge(df_ndvi[['point_id', 'DayOfYear', 'DetailedQA', 'EVI', 'NDVI', 'RelativeAzimuth', 
                    'SolarZenith', 'SummaryQA', 'ViewZenith', 'sur_refl_b01', 'sur_refl_b02', 
                    'sur_refl_b03', 'sur_refl_b07']], on='point_id', how='left') \
    .merge(df_lst[['point_id', 'lst_celsius']], on='point_id', how='left') \
    .merge(df_smap[['point_id', 'soil_moisture_am', 'soil_moisture_pm']], on='point_id', how='left') \
    .merge(df_worldpop[['point_id', 'pop_density']], on='point_id', how='left')

# Add original water point data with temporal information
df_final = kenya_sample[['wpdx_id', 'lat_deg', 'lon_deg', 'report_date', 'status_clean', 
                        'water_source_clean', 'water_tech_clean', 'management_clean']] \
    .merge(df_merged, left_on='wpdx_id', right_on='wpdx_id', how='inner')

df_final.to_csv("final_wpdx_environmental_data.csv", index=False)
print(f"Final merged dataset: {len(df_final)} points with {len(df_final.columns)} features")
print(f"Temporal range: {df_final['report_date'].min()} to {df_final['report_date'].max()}")
print(f"Missing values:\n{df_final.isnull().sum()}")
df_final.head()